[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/veselm73/SU2/blob/main/notebooks/SU2_StarDist_inference.ipynb)

# Cell Detection - Model Benchmark

**Author:** Mateusz Vesel  
**Task:** Compare 3 trained StarDist ensembles on held-out validation data

---

## What This Notebook Does

1. **Automatic benchmark** on 10 held-out validation frames (58-67)
2. **Compare all 3 models** and determine the best performer
3. **Optional:** Run inference on custom data with the best model

---

## Available Pre-trained Models

| Model | Epochs | Augmentation | N_Rays |
|-------|--------|--------------|--------|
| `100e_noaug_32rays` | 100 | No | 32 |
| `100e_noaug_64rays` | 100 | No | 64 |
| `120e_aug_32rays` | 120 | Yes | 32 |

---

**Just run all cells** - everything downloads automatically from GitHub.

## 1. Setup

In [ ]:
# Install dependencies
!pip install uv -q
!uv pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121 --system -q
!uv pip install "numpy<2" cellseg-models-pytorch pytorch-lightning tifffile scipy --system -q

In [ ]:
import sys
import os
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if not Path('/content/SU2').exists():
        !git clone https://github.com/veselm73/SU2.git /content/SU2
    else:
        !cd /content/SU2 && git pull
    os.chdir('/content/SU2')
    repo_root = Path('/content/SU2')
else:
    repo_root = Path('.').resolve()
    if repo_root.name == 'notebooks':
        repo_root = repo_root.parent

print(f"Repository: {repo_root}")

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm
from skimage.measure import regionprops
from scipy.spatial.distance import cdist
import urllib.request
import json

from cellseg_models_pytorch.postproc.functional.stardist.stardist import post_proc_stardist

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

AVAILABLE_MODELS = {
    "100e_noaug_32rays": "100 epochs, no aug, 32 rays",
    "100e_noaug_64rays": "100 epochs, no aug, 64 rays",
    "120e_aug_32rays": "120 epochs, with aug, 32 rays"
}

print(f"\nModels to benchmark: {list(AVAILABLE_MODELS.keys())}")

## 2. Download Validation Data & Model Weights

In [ ]:
# Download validation frames and annotations from GitHub
BASE_URL = "https://raw.githubusercontent.com/veselm73/SU2/main"

# Paths
VAL_DIR = repo_root / "data" / "val_annotations"
FRAMES_DIR = VAL_DIR / "frames"
ANNOTATIONS_CSV = VAL_DIR / "ensemble_sanity_check.csv"

VAL_DIR.mkdir(parents=True, exist_ok=True)
FRAMES_DIR.mkdir(parents=True, exist_ok=True)

# Download annotations
if not ANNOTATIONS_CSV.exists():
    print("Downloading annotations...")
    urllib.request.urlretrieve(f"{BASE_URL}/data/val_annotations/ensemble_sanity_check.csv", ANNOTATIONS_CSV)

# Download validation frames (58-67)
VALIDATION_FRAMES = list(range(58, 68))
print(f"Downloading {len(VALIDATION_FRAMES)} validation frames...")
for frame_idx in VALIDATION_FRAMES:
    frame_path = FRAMES_DIR / f"frame_{frame_idx:04d}.png"
    if not frame_path.exists():
        url = f"{BASE_URL}/data/val_annotations/frames/frame_{frame_idx:04d}.png"
        urllib.request.urlretrieve(url, frame_path)
        print(f"  Downloaded frame_{frame_idx:04d}.png")

# Load annotations
gt_df = pd.read_csv(ANNOTATIONS_CSV)
print(f"\nLoaded {len(gt_df)} annotations across frames {VALIDATION_FRAMES}")
print(f"Annotations per frame: ~{len(gt_df) // len(VALIDATION_FRAMES)}")

In [ ]:
def download_model_weights(model_name):
    """Download model weights from GitHub if not present."""
    weights_dir = repo_root / "weights" / model_name
    models_dir = weights_dir / "models"
    
    if models_dir.exists() and len(list(models_dir.glob("*.pth"))) == 5:
        return weights_dir
    
    print(f"Downloading {model_name} weights...")
    weights_dir.mkdir(parents=True, exist_ok=True)
    models_dir.mkdir(parents=True, exist_ok=True)
    
    base_url = f"{BASE_URL}/weights/{model_name}"
    
    for cfg in ["model_config.json", "inference_config.json"]:
        urllib.request.urlretrieve(f"{base_url}/{cfg}", weights_dir / cfg)
    
    for fold in range(1, 6):
        print(f"  Downloading fold_{fold}.pth (~53MB)...")
        urllib.request.urlretrieve(f"{base_url}/models/fold_{fold}.pth", models_dir / f"fold_{fold}.pth")
    
    return weights_dir

# Pre-download all model weights
print("Downloading model weights (this may take a few minutes)...\n")
for model_name in AVAILABLE_MODELS:
    download_model_weights(model_name)
print("\nAll weights downloaded!")

## 3. Model & Inference Functions

In [ ]:
import pytorch_lightning as pl
from cellseg_models_pytorch.models.stardist.stardist import StarDist
import torch.nn as nn

class StarDistModel(pl.LightningModule):
    def __init__(self, n_rays=32, encoder_name="resnet18", dropout=0.0):
        super().__init__()
        self.n_rays = n_rays
        wrapper = StarDist(
            n_nuc_classes=1, n_rays=n_rays, enc_name=encoder_name,
            model_kwargs={"encoder_kws": {"in_chans": 1}}
        )
        self.model = wrapper.model
    
    def forward(self, x):
        return self.model(x)


def load_ensemble(model_name):
    """Load 5-fold ensemble for a model."""
    weights_dir = repo_root / "weights" / model_name
    with open(weights_dir / "model_config.json") as f:
        config = json.load(f)
    with open(weights_dir / "inference_config.json") as f:
        inf_config = json.load(f)
    
    models = []
    for fold in range(1, 6):
        model = StarDistModel(n_rays=config['n_rays'], encoder_name=config['encoder_name'])
        model.load_state_dict(torch.load(weights_dir / "models" / f"fold_{fold}.pth", map_location='cpu'))
        model.eval().to(DEVICE)
        models.append(model)
    
    return models, inf_config


def preprocess(frame):
    """Percentile normalization."""
    frame = frame.astype(np.float32)
    p1, p99 = np.percentile(frame, (1, 99.8))
    return np.clip((frame - p1) / (p99 - p1 + 1e-8), 0, 1)


def detect(models, frame, prob_thresh, nms_thresh):
    """Ensemble detection."""
    x = torch.from_numpy(frame).float().unsqueeze(0).unsqueeze(0).to(DEVICE)
    
    all_stardist, all_prob = [], []
    with torch.no_grad():
        for model in models:
            out = model(x)['nuc']
            all_stardist.append(out.aux_map.cpu().numpy()[0])
            all_prob.append(torch.sigmoid(out.binary_map).cpu().numpy()[0, 0])
    
    try:
        labels = post_proc_stardist(
            np.mean(all_prob, axis=0), np.mean(all_stardist, axis=0),
            score_thresh=prob_thresh, iou_thresh=nms_thresh
        )
        return [(p.centroid[1], p.centroid[0]) for p in regionprops(labels)]
    except:
        return []

## 4. Benchmark All Models

In [ ]:
def benchmark_model(model_name, gt_df, frames_dir):
    """Benchmark a model on validation frames."""
    models, inf_config = load_ensemble(model_name)
    prob_thresh = inf_config['prob_thresh']
    nms_thresh = inf_config['nms_thresh']
    
    total_tp, total_fp, total_fn = 0, 0, 0
    
    for frame_idx in tqdm(VALIDATION_FRAMES, desc=model_name):
        # Load frame
        img = np.array(Image.open(frames_dir / f"frame_{frame_idx:04d}.png"))
        frame = preprocess(img)
        
        # Detect
        detections = detect(models, frame, prob_thresh, nms_thresh)
        
        # Get GT
        frame_gt = gt_df[gt_df['frame'] == frame_idx][['x', 'y']].values
        
        # Match (greedy, 5px threshold)
        if len(detections) > 0 and len(frame_gt) > 0:
            dist = cdist(np.array(detections), frame_gt)
            tp = 0
            for _ in range(min(len(detections), len(frame_gt))):
                min_idx = np.unravel_index(np.argmin(dist), dist.shape)
                if dist[min_idx] <= 5.0:
                    tp += 1
                    dist[min_idx[0], :] = np.inf
                    dist[:, min_idx[1]] = np.inf
                else:
                    break
            fp = len(detections) - tp
            fn = len(frame_gt) - tp
        elif len(detections) > 0:
            tp, fp, fn = 0, len(detections), 0
        else:
            tp, fp, fn = 0, 0, len(frame_gt)
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Cleanup
    del models
    torch.cuda.empty_cache()
    
    # Metrics
    deta = total_tp / (total_tp + total_fp + total_fn) if (total_tp + total_fp + total_fn) > 0 else 0
    prec = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    rec = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    
    return {'model': model_name, 'DetA': deta, 'Precision': prec, 'Recall': rec, 'TP': total_tp, 'FP': total_fp, 'FN': total_fn}

In [ ]:
# Run benchmark
print("="*60)
print("BENCHMARKING ALL MODELS ON HELD-OUT VALIDATION DATA")
print(f"Frames: {VALIDATION_FRAMES}")
print(f"Total GT annotations: {len(gt_df)}")
print("="*60 + "\n")

results = []
for model_name in AVAILABLE_MODELS:
    result = benchmark_model(model_name, gt_df, FRAMES_DIR)
    results.append(result)
    print(f"  {model_name}: DetA={result['DetA']:.4f}, Precision={result['Precision']:.4f}, Recall={result['Recall']:.4f}\n")

# Find best
best_idx = np.argmax([r['DetA'] for r in results])
BEST_MODEL = results[best_idx]['model']

print("="*60)
print(f"BEST MODEL: {BEST_MODEL} (DetA = {results[best_idx]['DetA']:.4f})")
print("="*60)

## 5. Results

In [ ]:
# Results table
results_df = pd.DataFrame(results)
results_df['DetA'] = results_df['DetA'].apply(lambda x: f"{x:.4f}")
results_df['Precision'] = results_df['Precision'].apply(lambda x: f"{x:.4f}")
results_df['Recall'] = results_df['Recall'].apply(lambda x: f"{x:.4f}")
print(results_df.to_string(index=False))

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
colors = ['#2ecc71', '#3498db', '#e74c3c']
models = [r['model'] for r in results]
metrics = [('DetA', [r['DetA'] if isinstance(r['DetA'], float) else float(r['DetA']) for r in results]),
           ('Precision', [r['Precision'] if isinstance(r['Precision'], float) else float(r['Precision']) for r in results]),
           ('Recall', [r['Recall'] if isinstance(r['Recall'], float) else float(r['Recall']) for r in results])]

# Recalculate from original results
results_raw = [benchmark_model(m, gt_df, FRAMES_DIR) for m in AVAILABLE_MODELS] if not isinstance(results[0]['DetA'], float) else results

for ax, (name, _) in zip(axes, metrics):
    values = [r[name] for r in results_raw] if results_raw else [float(r[name]) for r in results]
    bars = ax.bar(range(len(models)), values, color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels([m.replace('_', '\n') for m in models], fontsize=8)
    ax.set_ylabel(name)
    ax.set_ylim(0, 1)
    ax.set_title(name, fontweight='bold')
    for bar, v in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{v:.3f}', ha='center', fontsize=9)

plt.suptitle('Model Comparison on Held-Out Validation (10 frames)', fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Visualize Best Model Detections

In [ ]:
# Load best model
best_models, best_config = load_ensemble(BEST_MODEL)

# Visualize first 5 frames
fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for i, frame_idx in enumerate(VALIDATION_FRAMES[:5]):
    img = np.array(Image.open(FRAMES_DIR / f"frame_{frame_idx:04d}.png"))
    frame = preprocess(img)
    detections = detect(best_models, frame, best_config['prob_thresh'], best_config['nms_thresh'])
    frame_gt = gt_df[gt_df['frame'] == frame_idx][['x', 'y']].values
    
    # GT (top)
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].scatter(frame_gt[:, 0], frame_gt[:, 1], c='lime', s=40, marker='o', facecolors='none', linewidths=1.5)
    axes[0, i].set_title(f'Frame {frame_idx} - GT ({len(frame_gt)})')
    axes[0, i].axis('off')
    
    # Predictions (bottom)
    axes[1, i].imshow(img, cmap='gray')
    if detections:
        pred = np.array(detections)
        axes[1, i].scatter(pred[:, 0], pred[:, 1], c='red', s=40, marker='x', linewidths=1.5)
    axes[1, i].set_title(f'Frame {frame_idx} - Pred ({len(detections)})')
    axes[1, i].axis('off')

plt.suptitle(f'Best Model: {BEST_MODEL}\nTop: Ground Truth (green) | Bottom: Predictions (red)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

del best_models
torch.cuda.empty_cache()

---

## 7. (Optional) Run Inference on Custom Data

Upload your own `.tif` video to run inference with the best model.

In [ ]:
# Skip this cell if you don't want to run custom inference
RUN_CUSTOM_INFERENCE = False  # Set to True to enable

if RUN_CUSTOM_INFERENCE and IN_COLAB:
    from google.colab import files
    import tifffile
    
    print(f"Using best model: {BEST_MODEL}")
    print("Upload your .tif video:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        video = tifffile.imread(filename)
        print(f"\nLoaded {filename}: {video.shape}")
        
        # Load model
        models, config = load_ensemble(BEST_MODEL)
        roi = config.get('roi', {'x_min': 256, 'x_max': 512, 'y_min': 512, 'y_max': 768})
        
        # Run inference
        all_detections = []
        for frame_idx in tqdm(range(len(video)), desc="Detecting"):
            frame = video[frame_idx, roi['y_min']:roi['y_max'], roi['x_min']:roi['x_max']]
            frame = preprocess(frame)
            dets = detect(models, frame, config['prob_thresh'], config['nms_thresh'])
            for x, y in dets:
                all_detections.append({'frame': frame_idx, 'x': x + roi['x_min'], 'y': y + roi['y_min']})
        
        results_df = pd.DataFrame(all_detections)
        print(f"\nDetected {len(results_df)} cells across {len(video)} frames")
        
        # Save
        output_csv = filename.replace('.tif', '_detections.csv')
        results_df.to_csv(output_csv, index=False)
        files.download(output_csv)
        print(f"Saved to {output_csv}")
        
        del models
        torch.cuda.empty_cache()